# Case Study Model

## General Remarks

- Reverse Power Flow / Cost at substation
    - If the negative generation limit at the substation is not enforced, then substation has the option of pushing energy back towards the transmission system (reverse power flow)
    - We make the follwoing assumptions: 
        - reverse power flow is allowed at the substation 
        - for scheduled negative active power the substation is remunerated according to the market price (i.e. linear cost of energy provision at the substation) -> the substation has a linear cost function 
        - While the cost function will not have a quadratic termn, THERE WILL BE a penalty on $\alpha_0$ as it indicates how much the substation is willing to deviate from the schedule


## Notes and Todos:

- Verify the gamma decomposition
- Maybe check if 33bus gives nice/better results?
 
 
Done:   
- decide for a case! (combination of voltage limits and generators). Notes:
    - should there be an equal lambda case?
    - should the DERs be fully utilized in the deterministic case?
    - [Done]: Choosen Ex2 as set up
- put results handling in a function [done]
- remove unneccsray packages (e.g. gurobi from tomls) [done, but check on the "remains"]
- Remove quadratic term from substation [done]

In [3]:
# IJulia.qtconsole();

# set width of notbook 
display("text/html", "<style>.container { width:80% !important; }</style>")

In [4]:
# Set up environment from .toml-files
import Pkg
Pkg.activate(".")
# Pkg.instantiate()

# Load necessary packages
using DataFrames, CSV, LinearAlgebra, Dates # Basic Julia utility
using JLD, HDF5
using JuMP # Modeling Framework
using Mosek, MosekTools, Gurobi # Solver and Solver-Environment
using Logging # For some warnings in the functions

# Load functions and model
include("src/tools.jl") # Some additional functions
include("src/input.jl") # Type definitions and read-in functions
include("src/model_definition.jl") # Model definiton
include("src/output.jl") # Postprocessing of solved model

results_to_df (generic function with 1 method)

# 15Bus Case

- 15 bus case from Papavasilou paper
    - negative load is positive load
    - no line constraints
    - voltage at root is 1 p.u.
    
- Assumptions for all cases except mentioned differently
    - Two DERs one each at node 6 and 11 with `c1 = 10`, `c2 = 5`, `p_max = 1`, `q_max = 0.5`
    - No reverse power flow limit on substation
    - Substation pays `c1 = 50` for energy provision from the transmission network (no quadratic term) and a penalty of `200` on alpha to discourage high alpha

## Experiment 5

### Description

Ex5:
- tag: `experiment5` 
- all nodes have 20% standard deviation of their forecasted load
- all parameters as in AP's paper, including the negative load (i.e. uncontrollable generator with the same standard deviation
- EXCEPT: lines 2 and 3 also have s_max = 2 (instead of 0.256)
- Generation confidene: 95%, voltage confidence: 99%
- substation balancing participation penalty is set to 1000 (i.e. quadratic cost term)

Runs:
- `ex5_no_uncert`: deterministic case
- `ex5_uncert`: case with uncertainty
- `ex5_gen_uncert`: chance constraints only in generation


### Notes and Observations



In [12]:
experiment5 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex5_general_settings
ex5_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945, # 95%
    "z_v" => 2.576, # 99%
    "z_f" => 2.576, # 99%
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "toggle_thermal_cc" => false,
    "thermal_const_method" => 2, #1=soc, 2=inner approx
    "vfac" => 0, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex5_no_uncert:
ex5_no_uncert_settings = copy(ex5_general_settings)
ex5_no_uncert_settings["toggle_volt_cc"] = false
ex5_no_uncert_settings["toggle_gen_cc"] = false
ex5_no_uncert_settings["toggle_thermal_cc"] = false

experiment5["no_uncert"] = Dict(
    "verbose" => "no_uncert",
    "feeder" => feeder,
    "settings" => ex5_no_uncert_settings,
)

# ex5_uncert:
ex5_uncert_settings = copy(ex5_general_settings)
ex5_uncert_settings["toggle_volt_cc"] = true
ex5_uncert_settings["toggle_gen_cc"] = true
ex5_uncert_settings["toggle_thermal_cc"] = false
experiment5["uncert"] = Dict(
    "verbose" => "uncert",
    "feeder" => feeder,
    "settings" => ex5_uncert_settings,
)

# ex5_gen_uncert:
ex5_gen_uncert_settings = copy(ex5_general_settings)
ex5_gen_uncert_settings["toggle_volt_cc"] = false
ex5_gen_uncert_settings["toggle_gen_cc"] = true
ex5_gen_uncert_settings["toggle_thermal_cc"] = false
experiment5["gen_uncert"] = Dict(
    "verbose" => "gen_uncert",
    "feeder" => feeder,
    "settings" => ex5_gen_uncert_settings,
)

# Run 
experiment5 = run_experiment(experiment5)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment5)
    print(">>> Case $case")
    sleep(.2)
    display(experiment5[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.037558884 seconds
>>>> Post-Processing

Running gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.019324034 seconds
>>>> Post-Processing

Running no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.009955943 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,71.7329,1,1.1981,0.378944,1.0,0,0,0.465563,-50.0,-3.32827e-9,-19.9181,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.970112,0.5524,0.240744,0,-49.963,-4.43619,0.0,0.000534437,-0.018304,0,0,-1.98278e-7,-7.43554e-7,0.0,0,0.0,0.0,0.000255774,-50.0,-0.0369682,-2.77356e-11,0.0,-0.0369682,-0.0183041
3,0.0,3,0,0,0.984769,-0.2412,0.0552439,0,-19.9972,-8.76626,0.0,0.0477252,-1.63454,0,0,-2.71298e-7,-6.51034e-7,-66.6959,0,66.6959,-17.8678,0.000414416,-49.963,-6.1336,-3.10393,-12.5018,-3.2643,-1.63455
4,0.0,4,0,0,1.00731,-0.2412,0.0552439,0,-25.1136,-1.45394,0.0,0.121691,-4.16781,0,0,-1.04925e-6,-8.30509e-7,-2.44555e-6,0,1.29254e-6,3.92048e-7,0.000990722,-19.9972,-1.01732,-6.13372,2.74315e-7,-5.1164,-4.16784
5,0.0,5,0,0,1.00937,-0.0372927,-0.0498092,0,-25.3425,-1.12672,0.0,0.126728,-4.28435,0,0,-1.32613e-6,-8.19686e-7,-7.25137e-7,0,-5.14136e-7,7.94197e-8,0.00103784,-25.1136,-0.788289,-1.01723,5.55647e-8,-0.228939,-4.28437
6,0.0,6,0,0,1.01166,-0.0545927,-0.0541092,0,-25.5523,-0.825859,0.0,0.131343,-4.39113,0,0,-1.78832e-6,-7.90239e-7,0.0,0,0.0,0.0,0.0010757,-25.3425,-0.575798,-0.785559,0.0,-0.209761,-4.39115
7,0.0,7,0.105593,0.0669092,1.01979,-0.0836927,-0.0614092,0.263704,-26.13,-3.13706e-8,0.0,0.144053,-4.68522,-5.15303e-8,-37.69,-5.99317,-6.37813e-7,-1.07436e-6,0,8.23974e-7,1.66261e-7,0.00108905,-25.5523,-2.19458e-8,-0.577742,1.1631e-7,-0.577742,-4.68523
8,0.0,8,0,0,1.02077,-0.1969,0.0019,0,-27.4369,-1.86615,0.0,0.13271,-5.30265,0,0,-12.4909,-5.87498e-7,0.0,0,0.0,0.0,0.00106365,-26.1304,-1.30655,-1.59777e-7,0.0,-1.30655,-5.30269
9,0.0,9,0,0,1.01077,-0.224007,0.0966531,0,-26.1304,-2.2821e-7,0.0,0.13271,-4.66446,0,0,-2.03412e-6,-8.88834e-7,-7.13656e-6,0,6.77284e-6,-2.30366e-6,0.0011173,-25.1136,-1.5959e-7,-1.01676,-1.61098e-6,-1.01676,-4.66449


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,70.4343,1,1.1833,0.3237,1.0,0,0,0.428602,-50.0,-5.40272e-9,-19.8899,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-7.12378e-7,0.0,0,0,0,0,-3.32182e-7,-6.04708e-7,0.0,0,0.0,0.0,0,-50.0,-5.93648e-9,-4.50227e-11,0.0,-5.98151e-9,0
3,0.0,3,0,0,0.999807,-0.256,2.65741e-11,0,-6.16131,-11.7444,0.0,0,0,0,0,-4.19262e-7,-4.66164e-7,-43.8387,0,43.8387,-11.7444,0,-50.0,-8.21735,-4.98439e-7,-8.21735,-5.76295e-7,0
4,0.0,4,0,0,1.03464,-0.256,2.65741e-11,0,-37.6774,-3.82881e-7,0.0,0,0,0,0,-6.4309e-7,-3.41806e-7,-43.8387,0,43.8387,11.7444,0,-6.16131,-2.67901e-7,-8.21751,8.21751,-9.16258e-7,0
5,0.0,5,0,0,1.03576,-0.0461079,-0.0102331,0,-37.6774,-2.86456e-7,0.0,0,0,0,0,-6.56648e-7,-3.37952e-7,-1.63259e-6,0,-1.15351e-6,3.68639e-8,0,-37.6774,-2.00414e-7,-2.67877e-7,2.57913e-8,-4.16709e-8,0
6,0.0,6,0,0,1.03718,-0.0634079,-0.0145331,0,-37.6774,-2.47693e-7,0.0,0,0,0,0,-6.73747e-7,-3.33442e-7,0.0,0,0.0,0.0,0,-37.6774,-1.72695e-7,-1.9972e-7,0.0,-2.70258e-8,0
7,0.0,7,0.114408,0.0273331,1.04291,-0.0925079,-0.0218331,0.285719,-37.6774,-3.0918e-8,0.0,0,0,-1.44048e-7,-49.2583,-7.49022e-7,-3.17162e-7,-2.69938e-6,0,2.11952e-6,1.57265e-7,0,-37.6774,-2.16292e-8,-1.73278e-7,1.10017e-7,-4.16314e-8,0
8,0.0,8,0,0,1.05323,-0.1969,0.0019,0,-37.6774,-5.53377e-8,0.0,0,0,0,0,-8.82708e-7,0.0,0.0,0,0.0,0.0,0,-37.6774,-3.87438e-8,-1.31075e-7,0.0,-9.23312e-8,0
9,0.0,9,0,0,1.04354,-0.229992,0.00183313,0,-37.6774,-1.87214e-7,0.0,0,0,0,0,0.0,-3.20248e-7,-4.74601e-6,0,4.08618e-6,-2.75924e-8,0,-37.6774,-1.30921e-7,-2.67754e-7,-1.92957e-8,-1.56128e-7,0


>>> Case no_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,nu,delta_plus,delta_minus,mu_plus,mu_minus,eta_plus,eta_minus,eta_aP,eta_aQ,voltvar,lambda_anc,rx_pi_i,rx_pi_a,rx_etaQ,r_sum_mu_d,nu_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,61.5275,1,1.1833,0.3237,1.0,0,0,0,-50.0,-1.3543e-9,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.976936,0.5376,0.1855,0,-50.0,-1.98615e-7,0.0,0,0,0,0,-9.30013e-8,-1.68856e-7,0.0,0,0.0,0.0,0,-50.0,-1.65512e-9,-1.12858e-11,0.0,-1.66641e-9,0
3,0.0,3,0,0,0.999807,-0.256,1.95709e-11,0,-30.1628,-5.3144,0.0,0,0,0,0,-1.16915e-7,-1.30033e-7,-19.8372,0,19.8372,-5.3144,0,-50.0,-3.71839,-1.38967e-7,-3.71839,-1.6054e-7,0
4,0.0,4,0,0,1.03464,-0.256,1.95709e-11,0,-10.3255,-2.54011e-8,0.0,0,0,0,0,-1.78992e-7,-9.52812e-8,-19.8372,0,19.8372,5.3144,0,-30.1628,-1.77731e-8,-3.71847,3.71847,-2.55259e-7,0
5,0.0,5,0,0,1.0357,-0.0460628,-0.00820303,0,-10.3255,-1.03056e-8,0.0,0,0,0,0,-1.8256e-7,-9.42453e-8,-2.41589e-7,0,-1.07809e-7,-5.21328e-8,0,-10.3255,-7.21015e-9,-1.77715e-8,-3.64739e-8,-1.14922e-8,0
6,0.0,6,0,0,1.03708,-0.0633628,-0.012503,0,-10.3255,-3.63395e-10,0.0,0,0,0,0,-1.87093e-7,-9.30231e-8,0.0,0,0.0,0.0,0,-10.3255,-2.53363e-10,-7.18518e-9,0.0,-7.43855e-9,0
7,0.0,7,0.114363,0.025303,1.04267,-0.0924628,-0.019803,0,-10.3255,-5.7619e-8,0.0,0,0,0.0,-1.97491e-7,-2.07035e-7,-8.85744e-8,-7.55592e-7,0,5.94469e-7,4.09317e-8,0,-10.3255,-4.03082e-8,-2.54219e-10,2.86344e-8,-1.14196e-8,0
8,0.0,8,0,0,1.05334,-0.1969,0.0019,0,-10.3255,-1.14614e-7,0.0,0,0,0,0,-2.46517e-7,0.0,0.0,0,0.0,0.0,0,-10.3255,-8.02453e-8,-5.44596e-8,0.0,-2.57857e-8,0
9,0.0,9,0,0,1.04366,-0.230037,-0.000196966,0,-10.3255,-7.77846e-8,0.0,0,0,0,0,0.0,-8.92158e-8,-1.67111e-6,0,1.48677e-6,-1.01958e-8,0,-10.3255,-5.43958e-8,-1.77633e-8,-7.13006e-9,-4.37625e-8,0


In [11]:
save_experiment("experiment5", experiment5)

Saving uncert
Saving gen_uncert
Saving no_uncert
>>>> Saved with timestamp 190422_1211


In [8]:
m, meta = build_model(experiment5["uncert"]["feeder"], experiment5["uncert"]["settings"])
optimize!(m)

MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.


In [9]:
A = zeros(14, 12)
for b in 2:15
    for c in 1:12
        A[b-1, c] = shadow_price(m[:η][b,c])
    end
end

DataFrame(A)

,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,-1.09382e-6,-7.92339e-7,-1.06672e-6,0.0,0.0,0.0,0.0,0.0,0.0,-7.06097e-7,-5.444e-7,-9.6137e-7
2,0.0,0.0,0.0,0.0,-6.94047e-7,-1.00615e-6,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,-9.31583e-7,0.0,0.0,-9.61657e-7,-6.20009e-7,-7.93866e-7,0.0,0.0,0.0
4,-4.16429e-6,-3.9917e-6,-8.32058e-6,0.0,0.0,0.0,0.0,0.0,0.0,-4.5828e-6,-2.84213e-6,-3.72192e-6
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,-1.5629e-5,-4.75999e-5,-33.4991,-4.4011e-5,-9.56628e-6,-6.76036e-6,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,-1.4641e-5,-4.10641e-5,-34.4029,-5.02374e-5,-1.02132e-5,-7.01642e-6,0.0,0.0,0.0
9,-5.83368e-6,-5.09615e-6,-8.52663e-6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-3.78968e-6,-5.2592e-6


In [18]:
fed = experiment5["uncert"]["feeder"]
traverse(fed, 12)

1-element Array{Any,1}:
 12

## Experiment 1

We analyse the influence of the uncertainty in the system using the "standard" setup

### Description

Ex1:
- tag: `experiment1`
- tight bascase voltage (0.95-1.05) 
- all nodes have 20% standard deviation of their forecasted load

Runs:
- `ex1_no_uncert`: deterministic case
- `ex1_uncert`: case with uncertainty
- `ex1_gen_uncert`: Chance constraints only in generation


### Notes and Observations

- Recursive decomposition of prices for active power (eqtns (20) and (21)) are verified by the experimet
- Decompositon via voltage constraints (eqtns (23) and (24)) also verified by the experiment

In [36]:
experiment1 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_lv_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex1_general_settings
ex1_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)


# ex1_no_uncert:
ex1_no_uncert_settings = copy(ex1_general_settings)
ex1_no_uncert_settings["toggle_volt_cc"] = false
ex1_no_uncert_settings["toggle_gen_cc"] = false

experiment1["no_uncert"] = Dict(
    "verbose" => "ex1_no_uncert",
    "feeder" => feeder,
    "settings" => ex1_no_uncert_settings,
)

# ex1_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = true
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["uncert"] = Dict(
    "verbose" => "ex1_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# ex1_gen_uncert:
ex1_uncert_settings = copy(ex1_general_settings)
ex1_uncert_settings["toggle_volt_cc"] = false
ex1_uncert_settings["toggle_gen_cc"] = true
experiment1["gen_uncert"] = Dict(
    "verbose" => "ex1_gen_uncert",
    "feeder" => feeder,
    "settings" => ex1_uncert_settings,
)

# Run 
experiment1 = run_experiment(experiment1)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment1)
    display(experiment1[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_lv_noneg/
Running ex1_uncert
>>>> Building Model


KeyError: KeyError: key "toggle_thermal" not found

In [3]:
save_experiment("experiment1", experiment1)

Saving ex1_uncert
Saving ex1_gen_uncert
Saving ex1_no_uncert
>>>> Saved with timestamp 190320_0927


## Experiment 2

Same as Experiment1 but with wider voltage constraints

### Description

Ex2:
- tag: `experiment2` 
- bascase voltage (0.9-1.1) and `vfac = 0.02`: Resulting "real" voltage limits: ca. 0.91 - 1.09 (i.e. +- 9%)
- all nodes have 20% standard deviation of their forecasted load
- all parameters as in AP's paper, including the negative load (i.e. uncontrollable generator with the same standard deviation
- substation balancing participation penalty is set to 1000 (i.e. quadratic cost term)

Runs:
- `ex2_no_uncert`: deterministic case
- `ex2_uncert`: case with uncertainty
- `ex2_gen_uncert`: chance constraints only in generation


### Notes and Observations

- Without uncertainty the network is "uncongested" so that price for active power is the same at all nodes
- With uncertainty, some voltage constraints become binding and there are differences in the prices

In [8]:
experiment2 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_lv_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = abs.(loads) .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex2_general_settings
ex2_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "toggle_thermal_cc" => false,
    "thermal_const_method" => 0, #1=soc, 2=inner approx, other=off
    "vfac" => 0, # directly sets symetric voltage limit to min:(1-vfac) max:(1+vfac), use import values if vfac=0
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

# ex2_no_uncert:
ex2_no_uncert_settings = copy(ex2_general_settings)
ex2_no_uncert_settings["toggle_volt_cc"] = false
ex2_no_uncert_settings["toggle_gen_cc"] = false

experiment2["no_uncert"] = Dict(
    "verbose" => "no_uncert",
    "feeder" => feeder,
    "settings" => ex2_no_uncert_settings,
)

# ex2_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = true
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["uncert"] = Dict(
    "verbose" => "uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# ex2_gen_uncert:
ex2_uncert_settings = copy(ex2_general_settings)
ex2_uncert_settings["toggle_volt_cc"] = false
ex2_uncert_settings["toggle_gen_cc"] = true
experiment2["gen_uncert"] = Dict(
    "verbose" => "gen_uncert",
    "feeder" => feeder,
    "settings" => ex2_uncert_settings,
)

# Run 
experiment2 = run_experiment(experiment2)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment2)
    print(">>> Case $case")
    sleep(.2)
    display(experiment2[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_lv_noneg/
Running uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(86) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(92) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.020427029 seconds
>>>> Post-Processing


┌ Warning: Thermal constraint method 0 unknown. Proceeding with unconstrained lines.
└ @ Main C:\Users\Robert\Promotion\Workshop\02_cc_dlmp_varaware\01_casestudy_uncertan_dlmp\src\model_definition.jl:196



Running gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.003514463 seconds
>>>> Post-Processing

Running no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.002372737 seconds
>>>> Post-Processing



┌ Warning: Thermal constraint method 0 unknown. Proceeding with unconstrained lines.
└ @ Main C:\Users\Robert\Promotion\Workshop\02_cc_dlmp_varaware\01_casestudy_uncertan_dlmp\src\model_definition.jl:196
┌ Warning: Thermal constraint method 0 unknown. Proceeding with unconstrained lines.
└ @ Main C:\Users\Robert\Promotion\Workshop\02_cc_dlmp_varaware\01_casestudy_uncertan_dlmp\src\model_definition.jl:196


>>> Case uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,68.8929,1,0.9225,0.538211,1.0,0,0,0.886412,-50.0,0.0,-14.1406,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.950497,-0.0,-0.0,0,-50.4632,-55.5805,0.0,0.000113588,-0.322349,0,0,-3.995e-7,-303.693,3.35482e-8,-50.0,-0.463171,0.0,0.463171,-0.322174
3,0.0,3,0,0,0.969829,0.2768,0.400011,0,-37.729,-37.3805,0.0,0.0101434,-4.85402,0,0,-5.99329e-7,-1.22801e-5,3.43369e-5,-50.4632,-26.1545,-38.8887,-12.7342,-4.85388
4,0.0,4,0,0,0.999378,-0.5168,0.214511,0,-17.7696,-8.85481,0.0,0.0258639,-11.9569,0,0,-2.40509e-6,-3.65817e-6,0.00020377,-37.729,-6.19568,-26.155,-19.9594,-11.9568
5,0.0,5,0,0,1.00313,-0.5168,0.214511,0,-16.3753,-6.86193,0.0,0.026725,-12.4447,0,0,-3.297e-6,-2.75976e-6,0.000202796,-17.7696,-4.80084,-6.19513,-1.39429,-12.4447
6,0.0,6,0,0,1.00696,-0.372884,0.123142,0,-15.0979,-5.02965,0.0,0.0275139,-12.8916,0,0,-4.77645e-6,-2.12305e-6,0.00019498,-16.3753,-3.50673,-4.78422,-1.27749,-12.8917
7,0.0,7,0.441184,-0.106042,1.01932,-0.390184,0.118842,0.0450801,-11.5793,-2.20558e-7,0.0,0.0296867,-14.1227,-4.65357e-7,-1.62909e-7,-36.4996,-1.11122e-6,0.000151833,-15.0979,-1.54295e-7,-3.51857,-3.51857,-14.1229
8,0.0,8,0,0,0.990743,-0.419284,0.111542,0,-15.0198,-4.92234,0.0,0.0286522,-13.0504,0,0,-1.75891e-6,-1.42089,0.000235548,-14.8711,-3.4463,-3.29767,0.148625,-13.0502
9,0.0,9,0,0,1.00122,0.1969,0.0019,0,-14.8711,-4.71006,0.0,0.0286522,-13.0062,0,0,-3.59237e-6,-4.23753e-6,0.000237526,-17.7696,-3.2938,-6.19228,-2.89848,-13.006


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,52.7535,1,0.787837,0.543266,1.0,0,0,0.012358,-50.0,0.0,-1.57135,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.95,0.142137,0.405066,0,-50.4632,-55.5803,0.0,0,0,0,0,-1.56916e-6,-303.809,0,-50.0,-0.463169,0.0,0.463169,0
3,0.0,3,0,0,0.980882,-0.651463,0.219566,0,-37.7083,-37.3509,0.0,0,0,0,0,-2.24634e-6,-5.33077e-6,0,-50.4632,-26.1338,-38.8886,-12.7548,0
4,0.0,4,0,0,1.02742,-0.651463,0.219566,0,-17.7166,-8.77885,0.0,0,0,0,0,-6.75086e-6,-2.06033e-6,0,-37.7083,-6.14253,-26.1343,-19.9917,0
5,0.0,5,0,0,1.0318,-0.440129,0.14276,0,-16.3343,-6.80306,0.0,0,0,0,0,-8.35202e-6,-1.94428e-6,0,-17.7166,-4.75965,-6.14198,-1.38232,0
6,0.0,6,0,0,1.03618,-0.457429,0.13846,0,-15.0677,-4.9865,0.0,0,0,0,0,-1.09589e-5,-1.84005e-6,0,-16.3343,-3.47664,-4.74317,-1.26653,0
7,0.0,7,0.508429,-0.12566,1.05,-0.486529,0.13116,0.493818,-11.5794,-6.18925e-7,0.0,0,0,-6.1948e-6,-8.62687e-7,-36.1865,-1.57118e-6,0,-15.0677,-4.32978e-7,-3.48838,-3.48838,0
8,0.0,8,0,0,1.02254,0.1969,0.0019,0,-14.7831,-4.58403,0.0,0,0,0,0,-0.0,-2.205e-6,0,-14.7831,-3.20943,-3.20943,2.30643e-7,0
9,0.0,9,0,0,1.0327,-0.231434,0.0684061,0,-14.7831,-4.58403,0.0,0,0,0,0,-8.77745e-6,-0.0,0,-17.7166,-3.20567,-6.13916,-2.93349,0


>>> Case no_uncert

,objective,bus,gp,gq,voltage,fp,fq,alpha,lambda,pi,gamma,rho,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,49.9039,1,0.787837,0.543266,1.0,0,0,0,-50.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.95,0.142137,0.405066,0,-50.4783,-57.3968,0.0,0,0,0,0,-2.50115e-6,-313.738,0,-50.0,-0.478307,0.0,0.478307,0
3,0.0,3,0,0,0.980882,-0.651463,0.219566,0,-37.3066,-38.5716,0.0,0,0,0,0,-3.39152e-6,-9.77996e-6,0,-50.4783,-26.9879,-40.1596,-13.1717,0
4,0.0,4,0,0,1.02742,-0.651463,0.219566,0,-16.6615,-9.06576,0.0,0,0,0,0,-7.87707e-6,-3.60414e-6,0,-37.3066,-6.34328,-26.9884,-20.6451,0
5,0.0,5,0,0,1.0318,-0.438523,0.141637,0,-15.234,-7.02541,0.0,0,0,0,0,-1.01549e-5,-3.36612e-6,0,-16.6615,-4.91521,-6.34271,-1.4275,0
6,0.0,6,0,0,1.03618,-0.455823,0.137337,0,-13.9261,-5.14947,0.0,0,0,0,0,-1.42374e-5,-3.1574e-6,0,-15.234,-3.59027,-4.89819,-1.30792,0
7,0.0,7,0.506823,-0.124537,1.05,-0.484923,0.130037,0,-10.3237,-6.0948e-7,0.0,0,0,-1.51179e-6,-8.94904e-7,-37.3692,-2.64248e-6,0,-13.9261,-4.26371e-7,-3.60239,-3.60239,0
8,0.0,8,0,0,1.02254,0.1969,0.0019,0,-13.6321,-4.73385,0.0,0,0,0,0,-0.0,-4.00151e-6,0,-13.6321,-3.31433,-3.31433,4.18558e-7,0
9,0.0,9,0,0,1.0327,-0.23304,0.0695283,0,-13.6321,-4.73385,0.0,0,0,0,0,-6.49303e-6,-0.0,0,-16.6615,-3.31044,-6.3398,-3.02936,0


In [44]:
save_experiment("experiment2", experiment2)

Saving uncert
Saving gen_uncert
Saving no_uncert
>>>> Saved with timestamp 190320_1040


## Experiment 3

Same as before but instead of tighter voltage limits try higher load

In [10]:
experiment3 = Dict()

# Load feeder data 
feeder = load_feeder("data/feeder_data/basecase_noneg/")

# Set up
relative_std = 0.2
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

# ex3_general_settings
ex3_general_settings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945,
    "z_v" => 2.576,
    "toggle_volt_cc" => false,
    "toggle_gen_cc" => false,
    "vfac" => 0, # decrease upper bound by factor (1-vfac) and increase lower bound by (1+vfac)
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
    "loadfac" => 3
)

# ex3_no_uncert:
ex3_no_uncert_settings = copy(ex3_general_settings)
ex3_no_uncert_settings["toggle_volt_cc"] = false
ex3_no_uncert_settings["toggle_gen_cc"] = false

experiment3["no_uncert"] = Dict(
    "verbose" => "ex3_no_uncert",
    "feeder" => feeder,
    "settings" => ex3_no_uncert_settings,
)

# ex3_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = true
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["uncert"] = Dict(
    "verbose" => "ex3_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# ex3_gen_uncert:
ex3_uncert_settings = copy(ex3_general_settings)
ex3_uncert_settings["toggle_volt_cc"] = false
ex3_uncert_settings["toggle_gen_cc"] = true
experiment3["gen_uncert"] = Dict(
    "verbose" => "ex3_gen_uncert",
    "feeder" => feeder,
    "settings" => ex3_uncert_settings,
)

# Run 
experiment3 = run_experiment(experiment3)

# Show results
sleep(.2) # just to ensure correct printing order
for case in keys(experiment3)
    print(">>> Case $case")
    sleep(.2)
    display(experiment3[case]["results"])
end

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/
Running ex3_uncert
>>>> Building Model
>>>> Running Model
MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.
>>>> Model finished with status OPTIMAL in 0.006354567 seconds
>>>> Post-Processing

Running ex3_gen_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.002476817 seconds
>>>> Post-Processing

Running ex3_no_uncert
>>>> Building Model
>>>> Running Model
>>>> Model finished with status OPTIMAL in 0.00223549 seconds
>>>> Post-Processing

>>> Case uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.0676,1.0,1.0,-100.0,-7.68782e-10,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916252,0,-100.0,-0.000145897,0.0,-1.5754e-8,0,0,-3.26811e-6,-0.000764702,0.00363129,-100.0,-1.21581e-6,-6.40651e-12,1.1877e-6
3,0.0,3,0,0,0.950524,0,-100.0,-0.000106729,0.0,-5.32418e-6,0,0,-4.5928e-6,-5.34742e-5,0.0117104,-100.0,-7.46768e-5,-0.000102082,-2.95957e-5
4,0.0,4,0,0,1.00189,0,-99.9999,-2.54546e-5,0.0,-1.30342e-5,0,0,-1.39688e-5,-1.24571e-5,0.0211438,-100.0,-1.78105e-5,-7.46783e-5,-5.99181e-5
5,0.0,5,0,0,1.0105,0,-99.9999,-1.88947e-5,0.0,-1.3443e-5,0,0,-1.80855e-5,-9.65875e-6,0.0212127,-99.9999,-1.32194e-5,-1.78089e-5,-4.72767e-6
6,0.0,6,0,0,1.01952,0,-99.9999,-1.32239e-5,0.0,-1.37819e-5,0,0,-2.39752e-5,-7.56658e-6,0.0205562,-99.9999,-9.21983e-6,-1.31736e-5,-4.0367e-6
7,0.0,7,0.8,-0.0477213,1.04946,0,-99.9999,-2.49905e-7,0.0,-1.45891e-5,-88.4176,-2.06216e-6,-0.000103338,-4.4126e-6,0.0150349,-99.9999,-1.74825e-7,-9.25095e-6,-9.53643e-6
8,0.0,8,0,0,0.970333,0,-99.9999,-2.22249e-5,0.0,-1.47582e-5,0,0,-7.50706e-6,-5.08665e-5,0.0177988,-99.9999,-1.55604e-5,-1.08896e-5,4.53539e-6
9,0.0,9,0,0,1.00209,0,-99.9999,-1.55536e-5,0.0,-1.43558e-5,0,0,-1.67668e-5,-1.18817e-5,0.0209381,-99.9999,-1.08768e-5,-1.78007e-5,-7.42324e-6


>>> Case gen_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,431.923,1,3.8177,1.06751,1.0,1.0,-100.0,0.0,-31.6228,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.916265,0,-100.0,-1.46431e-5,0.0,0,0,0,-5.19691e-7,-7.75417e-5,0,-100.0,-1.22026e-7,0.0,1.22026e-7
3,0.0,3,0,0,0.950548,0,-100.0,-1.06024e-5,0.0,0,0,0,-6.95592e-7,-3.72078e-6,0,-100.0,-7.41829e-6,-1.02455e-5,-2.82723e-6
4,0.0,4,0,0,1.00193,0,-100.0,-3.07235e-6,0.0,0,0,0,-1.49108e-6,-9.49878e-7,0,-100.0,-2.14971e-6,-7.41843e-6,-5.26872e-6
5,0.0,5,0,0,1.01167,0,-100.0,-2.38737e-6,0.0,0,0,0,-1.87868e-6,-8.13124e-7,0,-100.0,-1.67029e-6,-2.14952e-6,-4.79231e-7
6,0.0,6,0,0,1.02172,0,-100.0,-1.81109e-6,0.0,0,0,0,-2.49437e-6,-7.01822e-7,0,-100.0,-1.26271e-6,-1.6645e-6,-4.01791e-7
7,0.0,7,0.8,-0.00573882,1.05434,0,-100.0,-4.76193e-7,0.0,0,-88.4189,-3.44537e-7,-1.01621e-5,-4.74925e-7,0,-100.0,-3.33128e-7,-1.26698e-6,-9.33847e-7
8,0.0,8,0,0,0.96786,0,-100.0,-2.67144e-6,0.0,0,0,0,-0.0,-1.95064e-6,0,-100.0,-1.87037e-6,-1.66633e-6,2.04037e-7
9,0.0,9,0,0,0.999696,0,-100.0,-2.38002e-6,0.0,0,0,0,-1.41964e-6,-0.0,0,-100.0,-1.66437e-6,-2.14853e-6,-4.84157e-7


>>> Case no_uncert

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,399.29,1,3.8177,1.07126,1.0,0,-100.0,0.0,0.0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.915774,0,-100.0,-7.20018e-6,0.0,0,0,0,-0.0,-3.84094e-5,0,-100.0,-6.00015e-8,0.0,6.00015e-8
3,0.0,3,0,0,0.949578,0,-100.0,-5.07783e-6,0.0,0,0,0,-2.51789e-7,-1.51122e-6,0,-100.0,-3.55287e-6,-5.03784e-6,-1.48497e-6
4,0.0,4,0,0,1.00027,0,-100.0,-1.25314e-6,0.0,0,0,0,-6.13942e-7,-0.0,0,-100.0,-8.76815e-7,-3.55294e-6,-2.67613e-6
5,0.0,5,0,0,1.01009,0,-100.0,-9.36669e-7,0.0,0,0,0,-7.81308e-7,-3.25326e-7,0,-100.0,-6.55326e-7,-8.76736e-7,-2.21411e-7
6,0.0,6,0,0,1.02022,0,-100.0,-6.68596e-7,0.0,0,0,0,-1.05106e-6,-2.82462e-7,0,-100.0,-4.66152e-7,-6.53056e-7,-1.86904e-7
7,0.0,7,0.8,-0.00323173,1.05305,0,-100.0,-3.86404e-8,0.0,0,-89.0495,-8.43607e-8,-4.76745e-6,-1.9593e-7,0,-100.0,-2.70315e-8,-4.67726e-7,-4.40694e-7
8,0.0,8,0,0,0.965763,0,-100.0,-9.90414e-7,0.0,0,0,0,-0.0,-7.79882e-7,0,-100.0,-6.93422e-7,-6.11847e-7,8.15756e-8
9,0.0,9,0,0,0.997666,0,-100.0,-8.739e-7,0.0,0,0,0,-6.07748e-7,-0.0,0,-100.0,-6.11129e-7,-8.76334e-7,-2.65205e-7


## Experiment 4

- Instead of uncertain load uncontrollable vs. controllable DERs
- Maybe better in the 33-bus system?

## Prototyping Space

In [5]:
res = experiment2["uncert"]["results"]

,objective,bus,gp,gq,voltage,alpha,lambda,pi,gamma,eta,delta_plus,delta_minus,mu_plus,mu_minus,voltvar,lambda_anc,rx_pi_i,rx_pi_a,r_sum_mu_d,eta_calc
,Float64,Any,Any,Any,Any,Any,Any,Any,Float64,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,42.1519,1,-0.0653151,0.933284,1.0,0.895129,-50.0,-3.34394e-9,-31.6735,0,0,0,0,0,0,0,0,0,0,0
2,0.0,2,0,0,0.900334,0,-50.137,-16.4401,0.0,-0.0512537,0,0,-7.25317e-8,-91.1244,0.000116804,-50.0,-0.137001,-2.78662e-11,0.137001,-0.352195
3,0.0,3,0,0,0.960447,0,-46.1416,-10.7297,0.0,-0.0596395,0,0,-1.34272e-7,-7.07984e-7,0.0139108,-50.137,-7.5074,-11.5028,-3.99543,-6.53764
4,0.0,4,0,0,1.04775,0,-39.8792,-1.7796,0.0,-0.0727832,0,0,-8.1725e-7,-1.40745e-7,0.0151213,-46.1416,-1.24518,-7.50755,-6.26237,-16.2326
5,0.0,5,0,0,1.05393,0,-39.599,-1.37908,0.0,-0.0729688,0,0,-1.00029e-6,-1.2926e-7,0.0138768,-39.8792,-0.96485,-1.24507,-0.280217,-16.6664
6,0.0,6,0,0,1.05993,0,-39.3422,-1.01084,0.0,-0.0731388,0,0,-1.28435e-6,-1.19442e-7,0.0126278,-39.599,-0.704766,-0.961509,-0.256743,-17.0639
7,0.0,7,0.753981,-0.224743,1.0781,0.0393694,-38.6351,-1.166e-7,0.0,-0.0736071,-27.0216,-4.12736e-8,-7.33553,-9.70597e-8,0.00925916,-39.3422,-8.15695e-8,-0.707145,-0.707145,-18.1587
8,0.0,8,0,0,1.06984,0,-37.0355,-2.28412,0.0,-0.0807118,0,0,-15.2886,-1.08976e-7,0.0127008,-38.6347,-1.59919,-1.66564e-7,-1.59919,-20.635
9,0.0,9,0,0,1.06031,0,-38.6347,-2.37903e-7,0.0,-0.0762531,0,0,-1.49742e-6,-1.19298e-7,0.0131609,-39.8792,-1.66369e-7,-1.2445,-1.2445,-18.1593


In [1]:
0.0005*50

0.025